In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [ ]:
# loading api_key
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [ ]:
# defining LLM
llm = ChatGroq(
    model = "gemma2-9b-it",
    temperature = 0.6,
    api_key = api_key
)

In [ ]:
# passing the prompt to LLM and getting reponse back
response = llm.invoke("I want to start a new Indian restaurant. Suggest a one fancy name, NO PERAMBLE.")
print(response.content)

Spice Zenith  



In [ ]:
# defining Prompt
name_prompt = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to start a new {cuisine} restaurant. Suggest a one fancy name, NO PERAMBLE."
)
name_prompt.format(cuisine = "Indian")

'I want to start a new Indian restaurant. Suggest a one fancy name, NO PERAMBLE.'

In [ ]:
# define chain - passing the prompt to LLM
name_chain = LLMChain(
    llm = llm,
    prompt = name_prompt
)

<ipython-input-7-95f58520ee68>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  name_chain = LLMChain(


In [ ]:
# running the chain
restaurant_name = name_chain.run("Indian")
print(restaurant_name)

<ipython-input-8-424a2825aa00>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  restaurant_name = name_chain.run("Indian")


Avani's Spice.  



In [ ]:
# Will pass the response from name_chain to another chain and get the menu itesm
# This is called "SimpleSequentialChain"
# input --> name_chain --> name (output) --> menu_chain --> menu (output)

In [ ]:
# creating menu_chain
menu_prompt = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu item for {restaurant_name} restaurant. Return it in comma seperated values. NO PERAMBLE."
)

In [ ]:
# creating menu_chain
menu_chain = LLMChain(
    llm = llm,
    prompt = menu_prompt
)

In [ ]:
# Createing Simple Sequential Chain
chain = SimpleSequentialChain(
    chains = [name_chain, menu_chain]
)

In [ ]:
# getting response
response = chain.run("Indian")
print(response)

Tandoori Chicken, Butter Chicken, Rogan Josh, Saag Paneer, Vegetable Korma, Lamb Vindaloo, Chicken Tikka Masala, Biryani, Naan, Samosas, Pakoras, Mango Lassi, Chai Tea.  



In [ ]:
# we didnt get restaurant name in the response from above. we only got menu
# for multiple output or multiple input we use SequentialChain

In [ ]:
# name chain
name_prompt = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to start a new {cuisine} restaurant. Suggest a one fancy name, NO PERAMBLE."
)
name_chain = LLMChain(
    llm = llm,
    prompt = name_prompt,
    output_key = "restaurant_name"
)

# menu chain
menu_prompt = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu item for {restaurant_name} restaurant. Return it in comma seperated values. NO PERAMBLE."
)
menu_chain = LLMChain(
    llm = llm,
    prompt = menu_prompt,
    output_key = "menu_items"
)

# Creating SequentialChain
chain = SequentialChain(
    chains = [name_chain, menu_chain],
    input_variables = ["cuisine"],
    output_variables = ["restaurant_name", "menu_items"]
)

In [ ]:
# running the chain
response = chain(["Indian"])
response
response = chain({"cuisine": "Indian"})
response

<ipython-input-16-f78affe29800>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain(["Indian"])


{'cuisine': 'Indian',
 'restaurant_name': 'Spice Odyssey. \n',
 'menu_items': 'Tandoori Chicken, Butter Chicken, Rogan Josh, Vindaloo, Saag Paneer, Chicken Tikka Masala, Biryani, Samosas, Naan, Pakoras, Mango Lassi, Chai  \n'}